In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import setup
import parse
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.activations import relu,linear
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv(setup.DATASET, nrows=setup.N_ROWS, dtype={"Fen": np.string_, "Evaluation": np.string_})
if setup.N_ROWS >= 469187 + 1:
    df = df.drop(469187) # Corrupted string in this dataset
    df = df.reset_index(drop=True)
df

### Vectorize the dataset

Each board is converted from FEN to the following vectorized representation:

| **# Features** | **Type**                 | **Value range** |
|----------------|--------------------------|-----------------|
| 64             | Piece                    | -6 - +6           |
| 1              | Side to move             | 0 / 1          |
| 4              | Castling rights          | 0 / 1          |
<!-- | 1              | En passant target square | 0-16            | -->
<!-- | 1              | Half-move clock          | 0-50            | -->

This results in a vector of 70 features. We discard the move counter and half-move clock provided in the last field in the FEN format. This means that we will have to take care of draws manually.

Un-normalized scores can be as high/low as 15k. Each score is converted to a range of -1 - 1 using a sigmoid to reflect the likelyhood of winning.

| **Output type**                 | **Value range** |
|----------------|--------------------------|
| Score             | 0 - 1                    |

In [ ]:
features = [f"f_{str(x)}" for x in range(1, setup.N_FEATURES+1)]
df_vectorized = pd.DataFrame(df["FEN"].apply(lambda fen_str: parse.fen_to_vector(fen_str)).to_list(), columns=features)
df_vectorized["label"] = df["Evaluation"].apply(lambda score: parse.normalize_stockfish_eval(score))
df_vectorized

In [ ]:
# Check for null values
assert df_vectorized[df_vectorized.isnull().values].empty
# Check shape (+1 if for the label)
assert df_vectorized.shape == (setup.N_ROWS, setup.N_FEATURES + 1)

In [ ]:
import setup

X = df_vectorized[features]
y = df_vectorized["label"]

# normalizer = tf.keras.layers.Normalization(axis=-1)
# normalizer.adapt(X)
'''
TODOs
* Try normalizing and check if we are doing it properly (axis=-1??)
* Try elu and leakyrelu
* Try SGD
'''

model = Sequential(
    [
        # normalizer,
        Input(shape=(setup.N_FEATURES,)),
        Dense(3048, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
        Dense(3048, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
        Dense(3048, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
        Dense(1024, activation=setup.HIDDEN_ACTIVATION, kernel_regularizer=tf.keras.regularizers.l2(setup.REGULARIZATION_RATE)),
        Dense(1, activation=setup.OUTPUT_ACTIVATION),
    ]
)
model.compile(
    loss=MeanSquaredError(),
    optimizer=Adam(learning_rate=setup.LEARNING_RATE),
    # optimizer=SGD(learning_rate=setup.LEARNING_RATE, nesterov=True, momentum=0.7),
)
model.summary()

In [ ]:
import setup

model.fit(X, y, epochs=setup.EPOCHS, batch_size=setup.BATCH_SIZE, callbacks=[
    tf.keras.callbacks.TerminateOnNaN(), 
    tf.keras.callbacks.EarlyStopping(monitor="loss", patience=setup.PATIENCE)
])

In [ ]:
model.save("models/loss_new")

### Test error on cross-validation

In [ ]:
# Use the next N_ROWS as cross-validation
# TODO use train_test_split!
df_cv = pd.read_csv(setup.DATASET, skiprows=setup.N_ROWS, nrows=setup.N_ROWS, names=["FEN", "Evaluation"], header=None)
df_cv

In [ ]:
features = [f"f_{str(x)}" for x in range(1, setup.N_FEATURES+1)]
df_cv_vectorized = pd.DataFrame(df_cv["FEN"].apply(lambda fen_str: parse.fen_to_vector(fen_str)).to_list(), columns=features)
df_cv_vectorized["label"] = df_cv["Evaluation"].apply(lambda score: parse.normalize_stockfish_eval(score))
df_cv_vectorized

In [ ]:
df_cv_vectorized["prediction"] = model.predict(df_cv_vectorized[features])
df_cv_vectorized["error"] = (df_cv_vectorized["prediction"] - df_cv_vectorized["label"])**2
df_cv_vectorized

In [ ]:
df_cv_vectorized["error"].mean()